# Laden der Brustkrebsdaten in die SQLite-Datenbank

In diesem Notebook werden die Brustkrebsdaten in die Datenbank geladen und danach für die Analyse wird ein neues Datenmodell erstellt.

In [203]:
#Löschen aller Variablen
%reset -f 

In [204]:
#Laden Libraries
import sqlite3
import pandas as pd
from google.colab import drive

In [205]:
#Version 
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)

In [206]:
sqlite3.version_info

(2, 6, 0)

In [207]:
pd.__version__

'1.3.5'

In [208]:
#Verbindung zu GoogleDrive
drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [209]:
%ls gdrive/Shareddrives/'Studie Breast Cancer'/breast_cancer/

BreastCancerDB.db  disease.xlsx         organizations.xlsx
careplans.csv      encounters.csv       patients.csv
careplans.xlsx     imaging_studies.csv  patients.xlsx
conditions.csv     immunizations.csv    payers.csv
conditions.xlsx    medications.csv      payer_transitions.csv
devices.csv        observations.csv     procedures.csv
devices.xlsx       observations.xlsx    providers.csv
disease.csv        organizations.csv    supplies.csv


In [210]:
# Liste mit Tabellen 
#!ls "/content/gdrive/Sha/Studie/breast_cancer/"
!ls "/content/gdrive/Shareddrives/Studie Breast Cancer/breast_cancer/"

BreastCancerDB.db  disease.xlsx		organizations.xlsx
careplans.csv	   encounters.csv	patients.csv
careplans.xlsx	   imaging_studies.csv	patients.xlsx
conditions.csv	   immunizations.csv	payers.csv
conditions.xlsx    medications.csv	payer_transitions.csv
devices.csv	   observations.csv	procedures.csv
devices.xlsx	   observations.xlsx	providers.csv
disease.csv	   organizations.csv	supplies.csv


In [211]:
# Verbindung zur Datenbank
#conn = sqlite3.connect('/content/gdrive/My Drive/Studie/BreastCancerDB.db')
conn = sqlite3.connect('/content/gdrive/Shareddrives/Studie Breast Cancer/BreastCancerDB.db')
cursor = conn.cursor()
print("Datenbank wurde erfolgreich geöffnet")

Datenbank wurde erfolgreich geöffnet


In [212]:
# Löschen Tabellen wenn sie schon exestieren
cursor.execute("DROP TABLE IF EXISTS patients")
cursor.execute("DROP TABLE IF EXISTS conditions")
cursor.execute("DROP TABLE IF EXISTS medications")
cursor.execute("DROP TABLE IF EXISTS observations")
cursor.execute("DROP TABLE IF EXISTS procedures")
cursor.execute("DROP TABLE IF EXISTS patient_careplans")
cursor.execute("DROP TABLE IF EXISTS patients_new")

Mit dem folgenden Code werden die Daten aus den CSV.-Dateien in die DataFrames von Pandas eingelesen.

In [213]:
# Laden Daten 
patients = pd.read_csv('/content/gdrive/My Drive/Studie/breast_cancer/patients.csv', sep=",")
conditions = pd.read_csv( '/content/gdrive/My Drive/Studie/breast_cancer/conditions.csv', sep=",")
medications = pd.read_csv('/content/gdrive/My Drive/Studie/breast_cancer/medications.csv', sep=",")
observations = pd.read_csv('/content/gdrive/My Drive/Studie/breast_cancer/observations.csv', sep=",")
procedures = pd.read_csv('/content/gdrive/My Drive/Studie/breast_cancer/procedures.csv', sep=",")

Die Tabellen enthalten die gleichen Namen. Den Zusammenhang zwischen den Daten ist mit dem folgenden ER-Diagramm visualisiert:  

<img src=https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectB/main/Daten_schema/Brustkrebsdaten.png>

Für die Daten werden neue Tabellen mit entsprechenden Attributen in der Datenbank eingelegt.

In [214]:
# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patients (
			Id nvarchar(36) primary key,
            BIRTHDATE Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            DRIVERS nvarchar(50),
            PASSPORT nvarchar(10),
            PREFIX nvarchar(4),
            FIRST nvarchar(50),
            LAST nvarchar(50),
            SUFFIX nvarchar(10),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(50),
            GENDER nvarchar(1),
            BIRTHPLACE nvarchar(50),
            ADDRESS nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS conditions (
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(256),
            CODE int,
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medications(
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observations (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            VALUE float,
            UNITS nvarchar(10),
            TYPE nvarchar(10),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedures(
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(256),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

Die erstellten Tabellen werden mit Daten aus Dataframes eingefügt.

In [215]:
# Einfügen der Daten in die erstellten Tabellen
patients.to_sql('patients', conn, if_exists='append', index=False)
conditions.to_sql('conditions', conn, if_exists='append', index=False)
medications.to_sql('medications', conn, if_exists='append', index=False)
observations.to_sql('observations', conn, if_exists='append', index=False)
procedures.to_sql('procedures', conn, if_exists='append', index=False)

# Sternschema erstellen

Wir benötigen eine Faktentabelle für unser Sternschema. Die Tabellen patients, medications, observations und procedures werden als Dimensionstabelle für das Sternschema genommen. Die neue Tabelle erhalten den Namen mit einem Präfix _new und sie werden mit den Daten aus der alten Tabellen befüllt.

## Dimensionstabellen 

In [216]:
# Dimensionstabellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patients_new(
			Id nvarchar(36) primary key,
            BIRTHDATE_YEAR Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            SUFFIX nvarchar(10),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(50),
            GENDER nvarchar(1),
            BIRTHPLACE nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS conditions_new (
            CONDITIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(256),
            CODE int,
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')             

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observations_new (
            OBSERVATIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            VALUE float,
            UNITS nvarchar(10),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')  

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedures_new(
            PROCEDURES_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(256),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''') 

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medications_new (
            MEDICATIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')                        

# Faktentabelle 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient_careplans (
            CAREPLAN_ID INTEGER PRIMARY KEY AUTOINCREMENT,
			PATIENT_ID nvarchar(36),
            CONDITIONS_ID int,
            CARE_TYPE nvarchar(20),
            TABLE_ID int,
            foreign key(PATIENT_ID) references patients_new(Id) 
            foreign key(CONDITIONS_ID) references conditions_new(CONDITIONS_ID)
			)
             ''')             

In dem nächsten Abschnitt werden die Daten aus den alten in die neuen Tabellen eingefügt.

In [217]:
cursor.execute('''
    INSERT INTO patients_new(Id, BIRTHDATE_YEAR, DEATHDATE,SSN,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,
    CITY,STATE, COUNTY,ZIP,HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE)
    SELECT  Id, strftime('%Y', BIRTHDATE) as BIRTHDATE_YEAR, DEATHDATE,SSN,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,
    CITY,STATE, COUNTY,ZIP, HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE 
    FROM patients
    ''') 

           
             
conn.commit()
cursor.execute(''' SELECT * FROM patients_new;''')
records = cursor.fetchall()

In [218]:
cursor.execute('''
    INSERT INTO conditions_new(START,STOP, PATIENT, ENCOUNTER,DESCRIPTION,CODE)
    SELECT START,STOP, PATIENT, ENCOUNTER,DESCRIPTION,CODE
    FROM conditions
    ''')
conn.commit()
cursor.execute(''' SELECT * FROM conditions_new;''')
records = cursor.fetchall()

In [219]:
cursor.execute('''
    INSERT INTO observations_new(DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, UNITS)
    SELECT DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, UNITS
    FROM observations
    ''')

conn.commit()
cursor.execute(''' SELECT * FROM observations_new;''')
records = cursor.fetchall()

In [220]:
cursor.execute('''
    INSERT INTO procedures_new(DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, BASE_COST, REASONCODE,REASONDESCRIPTION )
    SELECT DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, BASE_COST, REASONCODE,REASONDESCRIPTION
    FROM procedures
    ''')

conn.commit()
cursor.execute(''' SELECT * FROM procedures_new;''')
records = cursor.fetchall()

In [221]:
cursor.execute('''
    INSERT INTO medications_new(START, STOP, PATIENT, PAYER, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE,REASONDESCRIPTION )
    SELECT START, STOP, PATIENT, PAYER, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE,REASONDESCRIPTION 
    FROM medications
    ''')
conn.commit()
cursor.execute(''' SELECT * FROM medications_new;''')
records = cursor.fetchall()


In [222]:
# Ausgabe der daten aus der neuen Tablle patients_new
print(pd.read_sql_query('''
     SELECT *
     FROM patients_new
     ''', conn))

                                        Id  BIRTHDATE_YEAR DEATHDATE  \
0     d2061cc7-bee0-0e6c-3ac4-15c197c474e0            1956      None   
1     073d8e80-ff90-1c8d-57e4-29bfca52c87f            1964      None   
2     e1ff7e68-4097-9faf-514d-e4cfcfdf252e            1998      None   
3     a0f679cc-875f-dd72-ed13-9ca863ec6cf3            2004      None   
4     e4166a9c-f7c5-bef7-ea43-b96281a2d586            2006      None   
...                                    ...             ...       ...   
1014  2a41a25a-bb37-04db-4bd6-5a52474e309a            1991      None   
1015  a8bea34f-a8d3-f5e2-93d1-5e2e52307251            1952      None   
1016  8e1abff8-cf2e-ed15-1e93-753b6f2578ea            2002      None   
1017  f98799f2-c4da-761d-f3ac-5fac8e8fc8bc            1979      None   
1018  4d6c5c28-0195-8486-66bc-787fb1bc4099            1917      None   

              SSN SUFFIX         MAIDEN MARITAL   RACE    ETHNICITY GENDER  \
0     999-51-6528   None           None       M  white  n

## Faktentabelle

Mithilfe von Union und Join wird die Faktentabelle für das Datenmodell erstellt. Die Referenz zu jedem Primär-Schluss jeder Dimensionstabelle wird in die Faktentabelle gespeichert.
Zuerst wird die Tabellen patients_new und conditions_new mithilfe von Join verbunden. Das Ergebnis wird mit Tabellen medications_new, observations_new und procedures_new getrennt voneinander verbunden. Alle Ergebnisse werden anschließend mit Unions zusammengefügt. Die Faktentabelle ist mit dem folgenden ER-Diagramm abgebildet. 

<img src=https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectB/main/Daten_schema/Sternschema.png>

In [223]:
## Faktentabelle
cursor.execute('''
    INSERT INTO patient_careplans(PATIENT_ID, CONDITIONS_ID, CARE_TYPE, TABLE_ID)
    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Medications' as CARE_TYPE, medications_new.MEDICATIONS_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    medications_new ON patients_new.Id = medications_new.PATIENT
    
    UNION

    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Observations' as CARE_TYPE, observations_new.OBSERVATIONS_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    observations_new ON patients_new.Id =  observations_new.PATIENT

    UNION

    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Procedures' as CARE_TYPE, procedures_new.PROCEDURES_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    procedures_new ON patients_new.Id =  procedures_new.PATIENT

    ''')

conn.commit()
cursor.execute(''' SELECT * FROM patient_careplans;''')
records = cursor.fetchall()

Ausgabe der Daten aus der Faktentabelle:

In [224]:
print(pd.read_sql_query('''
     SELECT *
     FROM patient_careplans
     ''', conn))

      CAREPLAN_ID                            PATIENT_ID  CONDITIONS_ID  \
0               1  009121bf-a672-8942-443e-85e18a33f766             51   
1               2  009121bf-a672-8942-443e-85e18a33f766             51   
2               3  009121bf-a672-8942-443e-85e18a33f766             51   
3               4  009121bf-a672-8942-443e-85e18a33f766             51   
4               5  009121bf-a672-8942-443e-85e18a33f766             51   
...           ...                                   ...            ...   
8265         8266  fd6d5aae-f012-0e00-99bc-6668b73bf4f3             68   
8266         8267  fd6d5aae-f012-0e00-99bc-6668b73bf4f3             68   
8267         8268  fd6d5aae-f012-0e00-99bc-6668b73bf4f3             68   
8268         8269  fd6d5aae-f012-0e00-99bc-6668b73bf4f3             68   
8269         8270  fd6d5aae-f012-0e00-99bc-6668b73bf4f3             68   

         CARE_TYPE  TABLE_ID  
0     Observations      7190  
1     Observations      7191  
2     Observations

In [225]:
print(pd.read_sql_query('''
     SELECT *
     FROM patient_careplans
     WHERE CAREPLAN_ID<10
     ''', conn))

   CAREPLAN_ID                            PATIENT_ID  CONDITIONS_ID  \
0            1  009121bf-a672-8942-443e-85e18a33f766             51   
1            2  009121bf-a672-8942-443e-85e18a33f766             51   
2            3  009121bf-a672-8942-443e-85e18a33f766             51   
3            4  009121bf-a672-8942-443e-85e18a33f766             51   
4            5  009121bf-a672-8942-443e-85e18a33f766             51   
5            6  009121bf-a672-8942-443e-85e18a33f766             51   
6            7  009121bf-a672-8942-443e-85e18a33f766             51   
7            8  009121bf-a672-8942-443e-85e18a33f766             51   
8            9  009121bf-a672-8942-443e-85e18a33f766             51   

      CARE_TYPE  TABLE_ID  
0  Observations      7190  
1  Observations      7191  
2  Observations      7192  
3  Observations      7193  
4  Observations      7194  
5  Observations      7195  
6  Observations      7196  
7  Observations      7197  
8  Observations      7198  


In [226]:
print(pd.read_sql_query('''
     SELECT CARE_TYPE
     FROM patient_careplans
     GROUP BY CARE_TYPE
     ''', conn))

      CARE_TYPE
0   Medications
1  Observations
2    Procedures


In [227]:
print(pd.read_sql_query('''
     SELECT PATIENT_ID, CARE_TYPE
     FROM patient_careplans
     WHERE CARE_TYPE = 'Medications'
     ''', conn))

                               PATIENT_ID    CARE_TYPE
0    02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
1    02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
2    02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
3    02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
4    037404e1-0c87-534d-0fe2-e21ef20640f6  Medications
..                                    ...          ...
798  f4b7d173-38fd-e173-9242-aca2651c6b10  Medications
799  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
800  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
801  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
802  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications

[803 rows x 2 columns]


In [228]:
print(pd.read_sql_query('''
SELECT name FROM sqlite_master
WHERE type='table'
ORDER BY name;
     ''', conn))

                 name
0          conditions
1      conditions_new
2         medications
3     medications_new
4        observations
5    observations_new
6   patient_careplans
7            patients
8        patients_new
9          procedures
10     procedures_new
11    sqlite_sequence


In [229]:
conn.close()